## Extraindo uma playlist de um usuário

In [19]:
import os
import json
import pandas as pd
import numpy as np
import io
import requests
import subprocess 
from numpy import array

In [2]:
# generate authorization token
def generate_token():
    with open('auth.json') as json_file:  
            data = json.load(json_file)

    auth_64 = data["auth_id"]
    
    
    cmd = ('curl -X "POST" -H "Authorization: Basic {0}" -d grant_type=client_credentials https://accounts.spotify.com/api/token').format(auth_64)

    output = subprocess.check_output(cmd,shell=True)
    tmp = json.loads(output)
    
    return(tmp["access_token"])

# get the playlist id from playlist link
def playlist_extrator(link):
    link = link.split('/')
    username = ''
    playlist_id = ''
    for i in range(0,len(link)):
        if link[i] == 'user':
            username = link[i+1]
        elif link[i] == 'playlist':
            playlist_id = link[i+1]
        
    return (username,playlist_id)
# request the playlist info
def generate_request(auth,playlist_link):
    username,playlist_id = playlist_extrator(playlist_link)
    link = ('curl -X GET "https://api.spotify.com/v1/users/{0}/playlists/{1}/tracks?market=BR" -H "Accept: application/json" -H "Authorization: Bearer {2}"'.format(username,playlist_id,auth))
    return link




In [3]:
# lists the musics from a playlist
def list_music(auth,playlist_link):
    
    link  = generate_request(auth,playlist_link)
    output = subprocess.check_output(link,shell=True)
    obj = json.loads(output)
    music = ''
    try:
        for values in obj['tracks']['items']:
            if values['track']['id'] != None:
                music += values['track']['id'] +','
    except:

        for items in obj['items']:
            if items['track']['id'] != None:
                music += items['track']['id'] +','
                
    return music

# get dictionary of music from list of ids

def get_music_from_id(auth,music_list): 
    all_music_id = ('curl -X GET "https://api.spotify.com/v1/audio-features/?ids={0}" -H "Authorization: Bearer {1}"'.format(music,auth))   
    output = subprocess.check_output(all_music_id,shell=True)
    music_feat = json.loads(output)
    return music_feat

In [4]:
# get means from music from playlist 
def music_feat_fit(length,music_feat):

    feat = music_feat["audio_features"]
    feat_mean = dict()
    
    danceability = []
    energy = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    valence = []
    loudness = []
    tempo = []
    liveness = []
    
    if music_feat == False:
        
        feat_mean['danceability']       = "None"
        feat_mean['energy']             = "None"
        feat_mean['speechiness']        = "None"
        feat_mean['acousticness']       = "None"
        feat_mean['instrumentalness']   = "None"
        feat_mean['valence']            = "None"
        feat_mean['loudness']           = "None"
        feat_mean['tempo']              = "None"
        feat_mean['liveness']           = "None"

        return feat_mean 
    
    for music in feat:
        try:
            danceability.append(music['danceability'])
            energy.append(music['energy'])
            speechiness.append(music['speechiness'])
            acousticness.append(music['acousticness'])
            instrumentalness.append(music['instrumentalness'])
            valence.append(music['valence'])
            loudness.append(music['loudness'])
            tempo.append(music['tempo'])
            liveness.append(music['liveness'])
        except:
            pass
        
    danceability = pd.Series(danceability)
    energy = pd.Series(energy)
    speechiness = pd.Series(speechiness)
    acousticness = pd.Series(acousticness)
    instrumentalness = pd.Series(instrumentalness)
    valence = pd.Series(valence)
    loudness = pd.Series(loudness)
    tempo = pd.Series(tempo)
    liveness = pd.Series(liveness)
       
    feat_mean['danceability']       = np.mean(danceability)
    feat_mean['energy']             = np.mean(energy)
    feat_mean['speechiness']        = np.mean(speechiness) 
    feat_mean['acousticness']       = np.mean(acousticness)
    feat_mean['instrumentalness']   = np.mean(instrumentalness)
    feat_mean['valence']            = np.mean(valence)
    feat_mean['loudness']           = np.mean(loudness)
    feat_mean['tempo']              = np.mean(tempo)
    feat_mean['liveness']           = np.mean(liveness)
    
    return feat_mean 

In [5]:
# Link da tabela das resposta do form
with open('auth.json') as json_file:  
            data = json.load(json_file)

# Criar um dataframe pelo CSV da tabela      
url = data['url']
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))


## Analisando os atributos de cada playlist

In [6]:
auth = generate_token()

for i, trial in df.iterrows():

    playlist_link = (trial[3])
    music = list_music(auth,playlist_link)
    music_feat = get_music_from_id(auth, music)
    ind_mean = music_feat_fit(len(music_feat["audio_features"]),music_feat) 

    df.loc[i, "danceability"]      = str(ind_mean["danceability"])         
    df.loc[i, "energy"]            = str(ind_mean["energy"])   
    df.loc[i, "speechiness"]       = str(ind_mean["speechiness"])        
    df.loc[i, "acousticness"]      = str(ind_mean["acousticness"])         
    df.loc[i, "instrumentalness"]  = str(ind_mean["instrumentalness"])             
    df.loc[i, "valence"]           = str(ind_mean["valence"])
    df.loc[i, "loudness"]          = str(ind_mean["loudness"])
    df.loc[i, "tempo"]             = str(ind_mean["tempo"])
    df.loc[i, "liveness"]          = str(ind_mean["liveness"]) 
    print(((i+1)/len(df)*100),"%")


3.7037037037 %
7.40740740741 %
11.1111111111 %
14.8148148148 %
18.5185185185 %
22.2222222222 %
25.9259259259 %
29.6296296296 %
33.3333333333 %
37.037037037 %
40.7407407407 %
44.4444444444 %
48.1481481481 %
51.8518518519 %
55.5555555556 %
59.2592592593 %
62.962962963 %
66.6666666667 %
70.3703703704 %
74.0740740741 %
77.7777777778 %
81.4814814815 %
85.1851851852 %
88.8888888889 %
92.5925925926 %
96.2962962963 %
100.0 %


In [25]:
# Deletando informações pessoais (e-mail e link da playlist do spotify)
try:
    df_names = df['Nome Completo']
    del df ['Nome Completo']
    del df['Timestamp']
    del df['Email Address']
    del df['Link da sua playlist favorita do Spotify']
except:
    pass

df

,danceability,energy,speechiness,acousticness,instrumentalness,valence,loudness,tempo,liveness
0,0.7237916666666666,0.7855625,0.07103124999999999,0.07960545833333334,0.04932250208333336,0.48941666666666667,-5.365875000000002,122.18829166666666,0.22062916666666674
1,0.5287882352941176,0.7707176470588236,0.06866235294117648,0.06516938235294117,0.026957737294117646,0.45702588235294145,-5.629929411764705,124.38474117647063,0.23296823529411756
2,0.7437999999999998,0.7344099999999999,0.24014599999999997,0.18408440000000004,0.0016399424000000005,0.6551830000000001,-5.71197,111.13596,0.21512499999999998
3,0.48840217391304364,0.8203478260869568,0.14313043478260865,0.47356847826086956,2.402641304347826e-05,0.7603260869565219,-5.735260869565217,140.22523913043477,0.46871195652173925
4,0.6044583333333333,0.49978888888888867,0.07998194444444445,0.5232716666666666,0.007501033472222222,0.6309166666666665,-11.79668055555556,119.781,0.2760361111111111
5,0.51615,0.75121,0.054618999999999994,0.08561237400000002,0.041306650199999996,0.46897000000000005,-6.677139999999998,119.50043999999998,0.17766600000000005
6,0.63284,0.7304199999999996,0.06709300000000003,0.14247360000000003,0.011534239300000004,0.4700340000000001,-5.63649,120.20743,0.16813799999999993
7,0.6739299999999999,0.6741400000000002,0.093297,0.14503371999999992,0.004264186599999999,0.5137180000000001,-5.477929999999999,119.66069999999999,0.15602999999999997
8,0.8125299999999998,0.6101899999999999,0.17505000000000004,0.2867107000000001,0.0026332133999999994,0.6332599999999998,-3.4802000000000004,121.14699000000002,0.14753700000000003
9,0.6701506849315068,0.7475753424657532,0.15366301369863014,0.17852123287671232,0.0007574817808219177,0.5403123287671234,-5.904150684931506,117.27619178082192,0.21163424657534252


In [8]:
# Salvando base de dados para o excel
writer = pd.ExcelWriter('Respostas.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

##### Feature extraction|Test cluestering(teste/treinamento)

In [50]:
# from sklearn.cluster import KMeans
# import numpy as np
# X = np.array(df)
# kmeans = KMeans(n_clusters=5, random_state=0).fit(X)
# kmeans.labels_
# array([0, 0, 0, 1, 1, 1], dtype=np.int32)
# kmeans.predict([np.array(df.loc[2,:])])
# array([0, 1], dtype=np.int32)
# print(kmeans.cluster_centers_)
